## Problem Set 1: Take at Home  (45 points)

### Introduction
Typically when we train a Convolutional Neural Network as an end-to-end image classifier, we input an image to the network, which gets propagated through the network (feed forward).

We then obtain our posterior probabilities at the end of the network.

However, there is no “rule” that says we must allow the image to forward propagate through the entire network that includes the head. Instead, we can:

1) Stop propagation at a layer before the head of the network (such as an activation or pooling layer).

2) Extract the logits at this layer.

3) Treat the values as a feature vector.

Now these feature vectors can be utilized in other downstream tasks like classification. Our aim is to create a system where an input query image will be responded by a number of  images that have strong resemblance to the query image. This particular task is called **similarity search**. A naive way to perform this task, would be to compare images based on pixel values, patches, or some other high level feature taken from the image itself. You are askd to use the ResNet-50 architecture to produce features that can represent a concept aka a face with specific characteristics. 

### PS1.1 Loading of Dataset in Colab (5 points)

Create a jupyter notebook (eg on Google Colab) and download the LFW dataset, from [here](http://vis-www.cs.umass.edu/lfw/).

You can manually download the dataset using the above link and then upload to colab or altelnatively you can issue in colab the commands shown below


In [ ]:
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
!tar -xvf /content/lfw.tgz

### PS1.2 Using CNN for Feature Extraction (25 points)

Use ResNet50 to extract features vectors from raw images. You can use TF or Pytorch APIs to: 

* Obtain a ResNet-50  model pre-trained on a dataset such as ImageNet. 
* Perform necessary preprocessing on the images before feeding them into the network.
* Extract the features from the penultimate layer of the network (before the fully connected layer - the classification head).
* Store the features in a dictionary, where the key is the name of the image and the value is the feature vector.


In [ ]:
#1. Install the necessary libraries:
!pip install tensorflow

In [ ]:
#2. Import the required libraries:
import tensorflow as tf
import numpy as np
import os
import cv2

In [ ]:
#3. Load the pre-trained ResNet-50 model:
model = tf.keras.applications.ResNet50(
   include_top=False,
   weights='imagenet',
   input_shape=(224, 224, 3)
)

In [ ]:
#4. Define the function to preprocess the images:
def preprocess_image(image_path):
   image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
   image = tf.keras.preprocessing.image.img_to_array(image)
   image = tf.keras.applications.resnet50.preprocess_input(image)
   return image

In [ ]:
#5. Define a function to extract feature vectors from the ResNet-50 model:
def extract_features(image_path):
   image = preprocess_image(image_path)
   image = np.expand_dims(image, axis=0)
   features = model.predict(image)
   features = np.squeeze(features)
   return features

In [ ]:
#6. Define the directory where my images are stored and initialize an empty dictionary to store the features:
image_directory = 'C:\Users\andre\OneDrive\Documents\CS 670\lfw'
features_dict = {}

In [ ]:
#7. Iterate over the images in the directory, extract the features, and store them in the dictionary:
for filename in os.listdir(image_directory):
   if filename.endswith('.jpg') or filename.endswith('.png'):  # Update with specific image file extensions
       image_path = os.path.join(image_directory, filename)
       features = extract_features(image_path)
       features_dict[filename] = features

In [ ]:
# Now I have a dictionary (`features_dict`) where the key is the name of the image file, and 
# the value is the corresponding feature vector extracted from the ResNet-50 model.

 

### PS 1.3 Retrieving most similar images (15 points)

Use a nearest neighbor algorithm such as [this](https://scikit-learn.org/stable/modules/neighbors.html) to obtain the 10 most similar images to 5 query images of your choice. Choose the results that best illustrate the effectiveness of your system. 


In [ ]:
# To find the 10 most similar images to 5 query images using a nearest neighbor algorithm, I can 
# utilize the scikit-learn library in Python. Here's an example using the K-Nearest Neighbors algorithm:


In [ ]:
#1. Install scikit-learn:
!pip install scikit-learn

In [ ]:
#2. Import the necessary libraries:
import numpy as np
import os
from sklearn.neighbors import NearestNeighbors
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
#3. Define the directory where the feature vectors are stored:
feature_vectors_directory = 'C:\Users\andre\OneDrive\Documents\CS 670\NNeighbor'

In [ ]:
#4. Load the feature vectors from the directory into a list:
feature_vectors = []
filenames = []
for filename in os.listdir(feature_vectors_directory):
   feature_vector = np.load(os.path.join(feature_vectors_directory, filename))
   feature_vectors.append(feature_vector)
   filenames.append(filename)

In [ ]:
#5. Convert the list of feature vectors into a numpy array:
feature_vectors = np.array(feature_vectors)

In [ ]:
#6. Create a NearestNeighbors object and fit it with the feature vectors:
n_neighbors = 10
nn = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
nn.fit(feature_vectors)

In [ ]:
#7. Define the directory where the query images are stored:
query_images_directory = 'C:\Users\andre\OneDrive\Documents\CS 670\NNeighbor'

In [ ]:
#8. Iterate over the query images, extract their features, and find the nearest neighbors:
query_images = []
for filename in os.listdir(query_images_directory):
   image_path = os.path.join(query_images_directory, filename)
   image = Image.open(image_path).convert('RGB')
   image = preprocess_image(image_path)  
   query_images.append(image)
query_images = np.array(query_images)
# Extract features from the query images
query_features = []
for query_image in query_images:
   query_image = np.expand_dims(query_image, axis=0)
   with torch.no_grad():
       query_feature = model(query_image)
   query_feature = torch.squeeze(query_feature).numpy()
   query_features.append(query_feature)
query_features = np.array(query_features)
# Find the nearest neighbors for each query image
nearest_neighbors = nn.kneighbors(query_features, return_distance=False)

In [ ]:
#9. Visualize the results:
for i, query_image in enumerate(query_images):
   fig, axes = plt.subplots(1, n_neighbors + 1, figsize=(12, 6))
   axes[0].imshow(query_image.permute(1, 2, 0))
   axes[0].set_title('Query Image')
   axes[0].axis('off')
   for j, neighbor_idx in enumerate(nearest_neighbors[i]):
       neighbor_image_path = os.path.join(feature_vectors_directory, filenames[neighbor_idx])
       neighbor_image = Image.open(neighbor_image_path)
       axes[j + 1].imshow(neighbor_image)
       axes[j + 1].set_title(f'Neighbor {j+1}')
       axes[j + 1].axis('off')
   plt.tight_layout()
   plt.show()

In [ ]:
# This code will display the query image on the left and the 10 most similar images on the right for
# each of the 5 query images. I can modify the paths and parameters according to my specific setup.

In [ ]:
# Approach to solving the ResNet50 question:
# 1. Obtain Feature Vectors: First, extract feature vectors from the pre-trained ResNet-50 model for 
# a dataset of images. This involves loading each image, preprocessing it according to the requirements of the model, and 
# passing it through the model to obtain the feature vectors.
# 2. Store Feature Vectors: Store the extracted feature vectors in a suitable data structure, such as a dictionary 
# or an array, where the key/index corresponds to the image name or identifier, and the value is the feature vector.
# 3. Nearest Neighbor Algorithm: Utilize a nearest neighbor algorithm, such as the one provided by 
# scikit-learn's NearestNeighbors, to build a search index based on the feature vectors. This involves fitting 
# the algorithm with the feature vectors obtained in the previous step.
# 4. Query Image Processing: Preprocess the query images in a similar manner as the dataset images, passing them
# through the pre-trained model to obtain feature vectors for each query image.
# 5. Find Nearest Neighbors: Use the nearest neighbor algorithm to find the indices or identifiers of 
# the 10 most similar images for each query image based on their feature vectors. This step returns the 
# nearest neighbor indices or identifiers.
# 6. Retrieve Similar Images: Retrieve the actual images corresponding to the nearest neighbor indices/identifiers and
# store them for further analysis or visualization.
# 7. Present Results: Display the query images alongside their respective 10 most similar images to demonstrate 
# the effectiveness of the system in finding visually similar images. 


In [ ]:
#  Detailed explanation for nearest neighbor algorithm question:
# In this solution, I first utilize scikit-learn's NearestNeighbors algorithm to find the most similar 
# images to a set of query images. 
# The process involves the following steps:
# 1. I start by importing the necessary libraries, including NumPy for array manipulation, os for file operations, 
# sklearn.neighbors for the NearestNeighbors algorithm, and PIL for image handling.
# 2. Next, I define the directory where the pre-computed feature vectors are stored. These feature vectors were 
# obtained by extracting the penultimate layer's activations of the ResNet-50 model for each image.
# 3. I iterate over the feature vector directory, loading each feature vector into a list and keeping track of the
# corresponding filenames.
# 4. The list of feature vectors is then converted into a NumPy array, which is a suitable format
# for scikit-learn's NearestNeighbors algorithm.
# 5. I create a NearestNeighbors object, specifying the number of nearest neighbors 
# to retrieve (in this case, 10) and the metric to measure similarity (in this case, Euclidean distance).
# 6. The NearestNeighbors object is fitted with the feature vectors, allowing it to build an internal representation 
# for efficient nearest neighbor search.
# 7. I define the directory where the query images are stored and iterate over them. For each query image, I extract
# its feature vector using the ResNet-50 model.
# 8. The extracted feature vectors of the query images are used to find the nearest neighbors in the pre-computed 
# feature vector dataset. The result is an array of indices corresponding to the nearest neighbors for each query image.
# 9. Finally, I visualize the results by displaying each query image alongside its 10 most similar images. We use 
# matplotlib to create a grid of subplots, where the first column contains the query images, and the subsequent
# columns display the nearest neighbor images.

# Overall, this solution showcases the process of using a pre-trained ResNet-50 model to extract 
# feature vectors from images, followed by employing a nearest neighbor algorithm to find similar images based 
# on those features. The visualization of the results provides a clear demonstration of the effectiveness of the 
# system in retrieving visually similar images to the given queries.

 